In [1]:
import torch
import torch.nn.functional as F

In [2]:
words = open("names.txt", "r").read().lower().replace(" ", "").replace("-", "").replace(".", "").splitlines()

In [3]:
words[:3]

['aaban', 'aabharan', 'aabhas']

In [4]:
len(words)

55691

In [5]:
chars = sorted(list(set("".join(words))))
stoi = {s: i+1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s, i in stoi.items()}

In [6]:
def build_dataset(words):
    block_size = 3
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

In [7]:
import random
random.seed(42)
random.shuffle(words)

n1 = int(len(words) * 0.8)
n2 = int(len(words) * 0.9)

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([404301, 3]) torch.Size([404301])
torch.Size([50562, 3]) torch.Size([50562])
torch.Size([50594, 3]) torch.Size([50594])


In [8]:
Xtr.shape, Ytr.shape

(torch.Size([404301, 3]), torch.Size([404301]))

In [9]:
g = torch.Generator().manual_seed(42)
C = torch.randn((27, 20), generator=g)
w1 = torch.randn((60, 200), generator=g)
b1 = torch.randn(200, generator=g)
w2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, w1, b1, w2, b2]

In [10]:
sum(p.nelement() for p in parameters)

18167

In [14]:
for p in parameters:
    p.requires_grad = True

In [60]:
for _ in range(20000):

    # minibatch
    ix = torch.randint(0, Xtr.shape[0], (100,))

    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, 60) @ w1 + b1)
    logit = h @ w2 + b2
    loss = F.cross_entropy(logit, Ytr[ix])
    # print(loss.data.item())

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    for p in parameters:
        p.data += -0.0000001 * p.grad

print(loss.data.item())

1.9991940259933472


In [61]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1, 60) @ w1 + b1)
logits = h @ w2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

tensor(1.8998, grad_fn=<NllLossBackward0>)

In [62]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 60) @ w1 + b1)
logits = h @ w2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

tensor(1.9332, grad_fn=<NllLossBackward0>)

In [63]:
# sample from the model
block_size = 3

for _ in range(10):
    out = []
    context = [0] * block_size  # initialize with all ...
    while True:
        emb = C[torch.tensor([context])]  # (1,block_size,d)
        h = torch.tanh(emb.view(1, -1) @ w1 + b1)
        logits = h @ w2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print(''.join(itos[i] for i in out))

archini.
athushanth.
yushkatra.
bhathanesh.
uddhana.
puth.
redharaaj.
kalaiyanth.
prajsucgara.
nirutilvent.
